In [ ]:
PROJECT_ID = "baseball-woba"
LOCATION = "us-central1"

In [ ]:
! gcloud config set project {PROJECT_ID}

In [ ]:
bucket_name = "bucket-name-placeholder"
bucket_uri = f"gs://{bucket_name}"

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

if bucket_name == "" or bucket_name is None or bucket_name == "bucket-name-placeholder":
    bucket_name = PROJECT_ID + "aip-" + timestamp
    bucket_uri = "gs://" + bucket_name

bucket_uri

In [ ]:
from google.cloud import storage 
client = storage.Client(project=PROJECT_ID)

bucket = client.create_bucket(bucket_name, location=LOCATION)

print("Bucket {} created.".format(bucket.name))

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=bucket_uri)

In [ ]:
import numpy as np
import pandas as pd

LABEL_COLUMN = "woba" 

df = pd.read_csv("data.csv")
df = df.drop(['last_name, first_name', 'player_id', 'year'], axis=1)

In [ ]:
df

In [ ]:
df_train = df.sample(frac=0.85, random_state=100)
df_predict = df[~df.index.isin(df_train.index)]

In [ ]:
from google.cloud import bigquery
bq_client = bigquery.Client(project=PROJECT_ID)

In [ ]:

bq_dataset_id = f"{PROJECT_ID}.dataset_id_unique"
bq_dataset = bigquery.Dataset(bq_dataset_id)
bq_client.create_dataset(bq_dataset, exists_ok=True)

In [ ]:
dataset = aiplatform.TabularDataset.create_from_dataframe(
    df_source=df_train,
    staging_path=f"bq://{bq_dataset_id}.table-unique",
    display_name="sample-baseball" 
)

In [ ]:
%%writefile task.py

import argparse
import numpy as np
import os

import pandas as pd
import tensorflow as tf

from google.cloud import bigquery
from google.cloud import storage

# Read environmental variables
training_data_uri = os.getenv("AIP_TRAINING_DATA_URI")
validation_data_uri = os.getenv("AIP_VALIDATION_DATA_URI")
test_data_uri = os.getenv("AIP_TEST_DATA_URI")

# Read args
parser = argparse.ArgumentParser()
parser.add_argument('--label_column', required=True, type=str)
parser.add_argument('--epochs', default=10, type=int)
parser.add_argument('--batch_size', default=10, type=int)
args = parser.parse_args()

# Set up training variables
LABEL_COLUMN = args.label_column

# See https://cloud.google.com/vertex-ai/docs/workbench/managed/executor#explicit-project-selection for issues regarding permissions.
PROJECT_NUMBER = os.environ["CLOUD_ML_PROJECT_ID"]
bq_client = bigquery.Client(project=PROJECT_NUMBER)


# Download a table
def download_table(bq_table_uri: str):
    # Remove bq:// prefix if present
    prefix = "bq://"
    if bq_table_uri.startswith(prefix):
        bq_table_uri = bq_table_uri[len(prefix) :]
        
    # Download the BigQuery table as a dataframe
    # This requires the "BigQuery Read Session User" role on the custom training service account.
    table = bq_client.get_table(bq_table_uri)
    return bq_client.list_rows(table).to_dataframe()

# Download dataset splits
df_train = download_table(training_data_uri)
df_validation = download_table(validation_data_uri)
df_test = download_table(test_data_uri)

def convert_dataframe_to_dataset(
    df_train: pd.DataFrame,
    df_validation: pd.DataFrame,
):
    df_train_x, df_train_y = df_train, df_train.pop(LABEL_COLUMN)
    df_validation_x, df_validation_y = df_validation, df_validation.pop(LABEL_COLUMN)

    y_train = tf.convert_to_tensor(np.asarray(df_train_y).astype("float32"))
    y_validation = tf.convert_to_tensor(np.asarray(df_validation_y).astype("float32"))

    # Convert to numpy representation
    x_train = tf.convert_to_tensor(np.asarray(df_train_x).astype("float32"))
    x_test = tf.convert_to_tensor(np.asarray(df_validation_x).astype("float32"))

    # Convert to one-hot representation
    num_species = len(df_train_y.unique())
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_species)
    y_validation = tf.keras.utils.to_categorical(y_validation, num_classes=num_species)

    dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    dataset_validation = tf.data.Dataset.from_tensor_slices((x_test, y_validation))
    return (dataset_train, dataset_validation)

# Create datasets
dataset_train, dataset_validation = convert_dataframe_to_dataset(df_train, df_validation)

# Shuffle train set
dataset_train = dataset_train.shuffle(len(df_train))

def create_model(num_features):
    # Create model
    Dense = tf.keras.layers.Dense
    model = tf.keras.Sequential(
        [
            Dense(
                100,
                activation=tf.nn.relu,
                kernel_initializer="uniform",
                input_dim=num_features,
            ),
            Dense(75, activation=tf.nn.relu),
            Dense(50, activation=tf.nn.relu),            
            Dense(25, activation=tf.nn.relu),
            Dense(3, activation=tf.nn.softmax),
        ]
    )
    
    # Compile Keras model
    optimizer = tf.keras.optimizers.RMSprop(lr=0.001)
    model.compile(
        loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer
    )
    
    return model

# Create the model
model = create_model(num_features=dataset_train._flat_shapes[0].dims[0].value)

# Set up datasets
dataset_train = dataset_train.batch(args.batch_size)
dataset_validation = dataset_validation.batch(args.batch_size)

# Train the model
model.fit(dataset_train, epochs=args.epochs, validation_data=dataset_validation)

tf.saved_model.save(model, os.getenv("AIP_MODEL_DIR"))

In [ ]:
JOB_NAME = "custom_job_unique"

EPOCHS = 10
BATCH_SIZE = 5

CMDARGS = [
    "--label_column=" + LABEL_COLUMN,
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
]

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name=JOB_NAME,
    script_path="task.py",
    container_uri="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest",
    requirements=["google-cloud-bigquery>=2.20.0", "db-dtypes", "protobuf<3.20.0"],
    model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest",
)

In [ ]:
MODEL_DISPLAY_NAME = "baseball_model_unique"
# Start the training and create your model
model = job.run(
    dataset=dataset,
    model_display_name=MODEL_DISPLAY_NAME,
    bigquery_destination=f"bq://{PROJECT_ID}",
    args=CMDARGS,
)

In [ ]:
dataset.delete()

This attempt didn't work. Trying a regression model

In [ ]:
bq_client._credentials

In [ ]:
# Training Dataset
TRAINING_INPUT_DATASET_ID = "baseball_training_unique"
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{TRAINING_INPUT_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created Dataset {bq_client.project}.{bq_dataset.dataset_id}")

In [ ]:
# Create Test dataset
PREDICTION_INPUT_DATASET_ID = "baseball_prediction_unique"
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created Dataset {bq_client.project}.{bq_dataset.dataset_id}")

In [ ]:
dataset = aiplatform.TabularDataset.create_from_dataframe(
    df_source=df_train,
    display_name="Baseball Train",
    staging_path=f"bq://{PROJECT_ID}.{TRAINING_INPUT_DATASET_ID}.table-unique"
    #staging path
    #df_source=df_train,
    #staging_path=f"bq://{bq_dataset_id}.table-unique",
    #display_name="sample-baseball" 
)

In [ ]:
training_job = aiplatform.AutoMLTabularTrainingJob(
    display_name="job_unique2",
    optimization_prediction_type='regression',
    optimization_objective='minimize-rmse',
)



In [ ]:
model = training_job.run(
    dataset=dataset,
    model_display_name="baseball-model",
    training_fraction_split=0.9,
    validation_fraction_split=0.05,
    test_fraction_split=0.05,
    budget_milli_node_hours=1000,
    disable_early_stopping=True,
    target_column=LABEL_COLUMN
)

In [ ]:
model_evaluations = model.list_model_evaluations()

model_evaluations = list(model_evaluations)[0]
print(model_evaluations)

In [ ]:
RESULTS_DATASET_ID = "baseball_results_unique"
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{RESULTS_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created dataset {bq_client.project}.{bq_dataset.dataset_id}")

In [ ]:
PREDICTION_RESULTS_DATASET_ID = f"{PROJECT_ID}.{RESULTS_DATASET_ID}"

batch_job = model.batch_predict(
    job_display_name="regression-prediction",
    bigquery_source=f"bq://{PROJECT_ID}.{TRAINING_INPUT_DATASET_ID}.table-unique",
    instances_format="bigquery",
    predictions_format="bigquery",
    bigquery_destination_prefix=f"bq://{PREDICTION_RESULTS_DATASET_ID}"
)

In [ ]:
print(f"Select * from `{PREDICTION_RESULTS_DATASET_ID}`")

In [ ]:
df = (
    bq_client.query(f"SELECT predicted_woba.value, woba, ((predicted_woba.value-woba)/woba)*100 as err FROM `{PREDICTION_RESULTS_DATASET_ID}.*`")
    .result()
    .to_dataframe()
)

print(df.head())

In [ ]:
df['err'].mean()

In [ ]:
dataset.delete()
model.delete()
training_job.delete()
batch_job.delete()